# Final Exam 2563 - Neural Network (Orthopedic Patients Problem)

This exam problem has an objective to develop a neural network model to classify patients as belonging to one out of three categories: Normal (0), Disk Hernia (1) or Spondylolisthesis (2) from six biomechanical attributes (features) derived from the shape and orientation of the pelvis and lumbar spine:

1. pelvic incidence
2. pelvic tilt
3. lumbar lordosis angle
4. sacral slope
5. pelvic radius
6. grade of spondylolisthesis

In [1]:
# used for manipulating directory paths
import os

# Scientific and vector computation for python
import numpy as np

# Plotting library
from matplotlib import pyplot

# Optimization module in scipy
from scipy import optimize

# library written for this exam
import utilsNN as utils

# tells matplotlib to embed plots within the notebook
%matplotlib inline

### We start this exam problem by first loading the dataset

In [2]:
# Load training dataset and test dataset

# Read tab separated data
data_train = np.loadtxt(os.path.join('Data', 'NNOrthopedicPatientsData_training.txt'))
data_test = np.loadtxt(os.path.join('Data', 'NNOrthopedicPatientsData_test.txt'))

# First six columns of data are features and the last column is the label.
# Matrix X contains six features while vector y contains the label.

X, y = data_train[:, 0:6], data_train[:, 6].astype(int)
X_test, y_test = data_test[:, 0:6], data_test[:, 6].astype(int)

m = y.size  # number of training examples

You have been provided with a set of initialized network parameters ($\Theta^{(1)}, \Theta^{(2)}$). These are stored in `SampleOrthopedicWeight1.txt` and `SampleOrthopedicWeight2.txt` which will be loaded in the next cell of this notebook into `Theta1` and `Theta2`. The parameters have dimensions that are sized for a neural network with **20 hidden units** in the second layer (hidden layer) and **3 output units** (corresponding to three patient categories) in the third layer (output layer).

In [3]:
# Load initiallized network parameters

Theta1 = np.loadtxt(os.path.join('Data', 'SampleOrthopedicWeight1.txt'))
Theta2 = np.loadtxt(os.path.join('Data', 'SampleOrthopedicWeight2.txt'))
print('Shape of Theta1 =', Theta1.shape)
print('Shape of Theta2 =', Theta2.shape)

# Unroll parameters 
# To unroll the matrix into vector (1-D array), we use `np.ravel()` 
nn_params = np.concatenate([np.ravel(Theta1), np.ravel(Theta2)])
initial_nn_params = nn_params

Shape of Theta1 = (20, 7)
Shape of Theta2 = (3, 21)


In [4]:
nn_params

array([ 1.16380949e-01,  6.98418999e-02,  2.45563573e-02,  1.01284469e-01,
       -8.71080718e-02,  8.36715528e-02, -6.33857537e-02,  4.63551132e-02,
       -8.20442980e-02, -7.17665089e-02,  6.48633031e-02, -6.83272813e-02,
       -2.21245953e-02, -7.60535483e-02, -9.82373545e-03,  3.38100964e-02,
        1.08184098e-02, -1.15067957e-02, -4.64111014e-02, -6.65563225e-02,
       -6.80657400e-02, -9.20714560e-02, -3.74703159e-02,  1.39646638e-02,
       -5.32309366e-02,  1.12973174e-01, -4.29636647e-03,  9.15016378e-02,
       -4.58161410e-02, -4.83393539e-02,  2.36853054e-02,  6.78081526e-02,
        5.91726874e-02,  5.63878720e-02, -1.12240952e-02,  2.21384164e-03,
       -6.56123521e-02,  1.04366735e-01, -9.61648508e-03, -4.01843327e-02,
        1.89899690e-02, -9.14676085e-02, -8.88336797e-02,  1.11441885e-01,
       -5.55739520e-02, -3.87953098e-02,  1.04044084e-01,  9.15493388e-02,
       -1.11053344e-01, -9.92191618e-02, -8.75259371e-02,  3.02835335e-02,
        3.79152561e-02,  

### Model representation

The neural network has 3 layers - an input layer, a hidden layer and an output layer. 

The inputs are 6 biomechanical attributes. 

The hidden layer has 15 neurons.

The outputs are 3 patient categories (0 to 2).

The training data was loaded into the variables `X` and `y` while the test data was loaded into the variables `X_test` and `y_test` above.

In [5]:
# Setup the parameters you will use for this exam problem by yourself!
input_layer_size  = 6
hidden_layer_size = 20
num_labels = 3

In [6]:
def sigmoid(z):
    """
    Compute sigmoid function given the input z.
    
    Parameters
    ----------
    z : array_like
        The input to the sigmoid function. This can be a 1-D vector 
        or a 2-D matrix. 
    
    Returns
    -------
    g : array_like
        The computed sigmoid function. g has the same shape as z, since
        the sigmoid is computed element-wise on z.
        
    Instructions
    ------------
    Compute the sigmoid of each value of z (z can be a matrix, vector or scalar).
    """
    # convert input to a numpy array
    z = np.array(z)
    
    # You need to return the following variables correctly 
    g = np.zeros(z.shape)

    # ====================== YOUR CODE HERE ======================
    g = 1 / (1 + np.exp(-z))
    # =============================================================
    return g

In [7]:
def sigmoidGradient(z):
    """
    Computes the gradient of the sigmoid function evaluated at z. 
    This should work regardless if z is a matrix or a vector. 
    In particular, if z is a vector or matrix, you should return
    the gradient for each element.
    
    Parameters
    ----------
    z : array_like
        A vector or matrix as input to the sigmoid function. 
    
    Returns
    --------
    g : array_like
        Gradient of the sigmoid function. Has the same shape as z. 
    
    Instructions
    ------------
    Compute the gradient of the sigmoid function evaluated at
    each value of z (z can be a matrix, vector or scalar).    

    """

    g = np.zeros(z.shape)

    # ====================== YOUR CODE HERE ======================
    g = sigmoid(z) * (1 - sigmoid(z))
    # ============================================================
    return g

In [8]:
def nnCostFunction(nn_params,
                   input_layer_size,
                   hidden_layer_size,
                   num_labels,
                   X, y, lambda_=0.0):
    
    

    # Reshape nn_params back into the parameters Theta1 and Theta2, the weight matrices
    # for our 2 layer neural network
    Theta1 = np.reshape(nn_params[:hidden_layer_size * (input_layer_size + 1)],
                        (hidden_layer_size, (input_layer_size + 1)))  # dimension = 25 X 401
    Theta2 = np.reshape(nn_params[(hidden_layer_size * (input_layer_size + 1)):],
                        (num_labels, (hidden_layer_size + 1)))  # dimension = 10 X 26
    # Setup some useful variables
    m = y.size
         
    # You need to return the following variables correctly 
    J = 0
    grad = []
    Theta1_grad = np.zeros(Theta1.shape)
    Theta2_grad = np.zeros(Theta2.shape)

    
    # ====================== Code for Section 1.3 Feedforward and cost function ======================
    a1 = np.concatenate([np.ones((m,1)),X],axis =1 ) # dimension = 5000 X 401
    
    z2 = np.dot(a1,Theta1.T) # dimension = 5000 X 25
    a2 = sigmoid(z2) # dimension = 5000 X 25
    # Add bias unit to a2
    a2 = np.concatenate([np.ones((a2.shape[0],1)),a2],axis=1) # 5000 X 26
    
    z3 = np.dot(a2,Theta2.T) # dimension = 5000 X 10
    a3 = sigmoid(z3) # dimension = 5000 X 10
    
    # values of the hypothesis h= activation unit lalues of the last (output) Layer
    h = a3 # dimension = 5000 X 10
    
    ### Cost Function ###
    #set y_matrix to be a matrix of (m x num_labels) where the element in each row that equals to 1 will be the 
    #(value of y + 1 ) th element in that row, for example, if the value of y in the 10th row = 2, y[9] = 2, then
    # the (2+1 = 3)rd element of the 10th row will be 1 while other elements are zeros.
    #i.e. [0., 0., 1., 0.,0.,0.,0.,0.,0.,0.]
    y_matrix = y
    y_matrix = np.eye(num_labels)[y_matrix] # dimension = 5000 X 10
    
    J = (-1 / m ) * np.sum((np.log(h) * y_matrix) + np.log(1 - h) * (1 - y_matrix))

    # ====================== End of Code for Section 1.3 Feedforward and cost function ======================
    #!!
    #!!
    # ====================== Code for Section 1.4 Regularized cost function ======================
    # Add regularization term to the cost function from Section 1.3
    # reg_term does not include the square of theta of bias units, 
    # For the matrices Theta1 and Theta2, this corresponds to the first column of each matrix.
    # Therefore we use [:, 1:] to square every theta except the first column of the matrix 
    # (the theta of bias units)
    
    reg_term = (lambda_ / (2*m)) * (np.sum(np.square(Theta1[:, 1:])) + np.sum(np.square(Theta2[:, 1:])))
    J = J + reg_term
    
    # ====================== End of Code for Section 1.4 Regularized cost function ======================
    #!!
    #!!                 
    # ====================== Code for Section 2.3 Backpropagation ======================
    delta_3 = h - y_matrix # dimension = 5000 X 10
    
    # To calculate delta_2, do not include weight form bias unit, therefore we use  Theta2[:, 1:]
    # (Not include the first column of Theta2 which corresponds to the weigth of bias unit)
    
    delta_2 = np.dot(delta_3, Theta2[:, 1:]) * sigmoidGradient(z2) # dimension =500 X 25
    
    Delta1 = np.dot(delta_2.T,a1) # dimension = 25 X 401 ---> The first column of Delta1 corresponds to the bias units
    Delta2 = np.dot(delta_3.T,a2) # dimension = 10 X 26 ---> The first column of Delta2 corresponds to the bias units
    
    Theta1_grad = (1/m)*Delta1 # dimension = 25 X 401
    Theta2_grad = (1/m)*Delta2 # dimension = 10 X 26
    
    # To untoll the matrix into vector (1-D array), we use np.ravel()
    
    grad = np.concatenate([np.ravel(Theta1_grad), np.ravel(Theta2_grad)])
    
    # ====================== End of Code for Section 2.3 Backpropagation ======================
    #!!
    #!!
    # ====================== Code for Section 2.5 Regularized Neural Network ======================
    
    # Add regularization to gradient
    Theta1_grad[:, 1:] = Theta1_grad[:, 1:] + (lambda_ / m) * Theta1[:, 1:]
    Theta2_grad[:, 1:] = Theta2_grad[:, 1:] + (lambda_ / m) * Theta2[:, 1:]
    
    # update grad with regularizatoin
    grad = np.concatenate([np.ravel(Theta1_grad), np.ravel(Theta2_grad)]) 
    # ====================== End of Code for Section 2.5 Regularized Neural Network ======================
    
    return J, grad

In [9]:
# Weight regularization parameter (we set this to 1 here).
lambda_ = 0
J, _ = nnCostFunction(nn_params, input_layer_size, hidden_layer_size,
                      num_labels, X, y, lambda_)

print('Cost at parameters (loaded from ex4weights): %.6f' % J)

lambda_ = 1
J, _ = nnCostFunction(nn_params, input_layer_size, hidden_layer_size,
                      num_labels, X, y, lambda_)

print('Cost at parameters (loaded from ex4weights): %.6f' % J)


Cost at parameters (loaded from ex4weights): 2.068831
Cost at parameters (loaded from ex4weights): 2.070556


In [10]:
pred = utils.predict(Theta1, Theta2, X)
print('Training Set Accuracy: %f' % (np.mean(pred == y) * 100))

Training Set Accuracy: 21.739130


In [11]:
pred = utils.predict(Theta1, Theta2, X_test)
print('Testing Set Accuracy: %f' % (np.mean(pred == y_test) * 100))

Training Set Accuracy: 12.500000


In [14]:
options= {'maxiter': 2000}

lambda_ = 0

# Create "short hand" for the cost function to be minimized
costFunction = lambda p: nnCostFunction(p, input_layer_size,
                                        hidden_layer_size,
                                        num_labels, X, y, lambda_)

# Now, costFunction is a function that takes in only one argument
# (the neural network parameters)
res = optimize.minimize(costFunction,
                        nn_params,
                        jac=True,
                        method='TNC',
                        options=options)

# get the solution of the optimization
nn_params_Final = res.x
        
# Obtain the optimal Theta1 and Theta2 back from nn_params
Theta1_Final = np.reshape(nn_params_Final[:hidden_layer_size * (input_layer_size + 1)],
                    (hidden_layer_size, (input_layer_size + 1)))

Theta2_Final = np.reshape(nn_params_Final[(hidden_layer_size * (input_layer_size + 1)):],
                    (num_labels, (hidden_layer_size + 1)))



C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:47: RuntimeWarning: divide by zero encountered in log
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:47: RuntimeWarning: invalid value encountered in multiply


In [15]:
utils.checkNNGradients(nnCostFunction, lambda_)

# Also output the costFunction debugging values
debug_J, _  = nnCostFunction(nn_params_Final, input_layer_size,
                          hidden_layer_size, num_labels, X, y, lambda_)

print('\n\nCost at (fixed) debugging parameters (w/ lambda = %f): %f ' % (lambda_, debug_J))


[[-9.27825235e-03 -9.27825236e-03]
 [-3.04978709e-06 -3.04978914e-06]
 [-1.75060084e-04 -1.75060082e-04]
 [-9.62660640e-05 -9.62660620e-05]
 [ 8.89911959e-03  8.89911960e-03]
 [ 1.42869450e-05  1.42869443e-05]
 [ 2.33146358e-04  2.33146357e-04]
 [ 1.17982666e-04  1.17982666e-04]
 [-8.36010761e-03 -8.36010762e-03]
 [-2.59383093e-05 -2.59383100e-05]
 [-2.87468729e-04 -2.87468729e-04]
 [-1.37149709e-04 -1.37149706e-04]
 [ 7.62813550e-03  7.62813551e-03]
 [ 3.69883257e-05  3.69883234e-05]
 [ 3.35320351e-04  3.35320347e-04]
 [ 1.53247082e-04  1.53247082e-04]
 [-6.74798369e-03 -6.74798370e-03]
 [-4.68759764e-05 -4.68759769e-05]
 [-3.76215583e-04 -3.76215587e-04]
 [-1.66560294e-04 -1.66560294e-04]
 [ 3.14544970e-01  3.14544970e-01]
 [ 1.64090819e-01  1.64090819e-01]
 [ 1.64567932e-01  1.64567932e-01]
 [ 1.58339334e-01  1.58339334e-01]
 [ 1.51127527e-01  1.51127527e-01]
 [ 1.49568335e-01  1.49568335e-01]
 [ 1.11056588e-01  1.11056588e-01]
 [ 5.75736494e-02  5.75736493e-02]
 [ 5.77867378e-02  5

In [16]:
options= {'maxiter': 2000}

lambda_ = 1

# Create "short hand" for the cost function to be minimized
costFunction = lambda p: nnCostFunction(p, input_layer_size,
                                        hidden_layer_size,
                                        num_labels, X, y, lambda_)

# Now, costFunction is a function that takes in only one argument
# (the neural network parameters)
res = optimize.minimize(costFunction,
                        nn_params,
                        jac=True,
                        method='TNC',
                        options=options)

# get the solution of the optimization
nn_params_Final = res.x
        
# Obtain the optimal Theta1 and Theta2 back from nn_params
Theta1_Final = np.reshape(nn_params_Final[:hidden_layer_size * (input_layer_size + 1)],
                    (hidden_layer_size, (input_layer_size + 1)))

Theta2_Final = np.reshape(nn_params_Final[(hidden_layer_size * (input_layer_size + 1)):],
                    (num_labels, (hidden_layer_size + 1)))



In [17]:
utils.checkNNGradients(nnCostFunction, lambda_)

# Also output the costFunction debugging values
debug_J, _  = nnCostFunction(nn_params_Final, input_layer_size,
                          hidden_layer_size, num_labels, X, y, lambda_)

print('\n\nCost at (fixed) debugging parameters (w/ lambda = %f): %f ' % (lambda_, debug_J))

[[-0.00927825 -0.00927825]
 [-0.00559136 -0.00559136]
 [-0.02017486 -0.02017486]
 [-0.00585433 -0.00585433]
 [ 0.00889912  0.00889912]
 [ 0.01315402  0.01315402]
 [-0.01049831 -0.01049831]
 [-0.01910997 -0.01910997]
 [-0.00836011 -0.00836011]
 [ 0.01976123  0.01976123]
 [ 0.00811587  0.00811587]
 [-0.01515689 -0.01515689]
 [ 0.00762814  0.00762814]
 [ 0.00827936  0.00827936]
 [ 0.02014747  0.02014747]
 [ 0.00315079  0.00315079]
 [-0.00674798 -0.00674798]
 [-0.0109273  -0.0109273 ]
 [ 0.01262954  0.01262954]
 [ 0.01809234  0.01809234]
 [ 0.31454497  0.31454497]
 [ 0.14895477  0.14895477]
 [ 0.17770766  0.17770766]
 [ 0.14745891  0.14745891]
 [ 0.15953087  0.15953087]
 [ 0.14381027  0.14381027]
 [ 0.11105659  0.11105659]
 [ 0.03839516  0.03839516]
 [ 0.0775739   0.0775739 ]
 [ 0.03592373  0.03592373]
 [ 0.07350885  0.07350885]
 [ 0.03392626  0.03392626]
 [ 0.0974007   0.0974007 ]
 [ 0.04486928  0.04486928]
 [ 0.05899539  0.05899539]
 [ 0.03843063  0.03843063]
 [ 0.06015138  0.06015138]
 

In [23]:
pred = utils.predict(Theta1_Final, Theta2_Final, X)
print(lambda_)
print('Training Set Accuracy: %f' % (np.mean(pred == y) * 100))

1
Training Set Accuracy: 93.478261


In [24]:
pred = utils.predict(Theta1_Final, Theta2_Final, X_test)
print(lambda_)
print('Test Set Accuracy: %f' % (np.mean(pred == y_test) * 100))

1
Training Set Accuracy: 91.250000
